In [ ]:
import pandas as pd
from tqdm.auto import tqdm
import random

In [ ]:
with open('/var/projetos/Jupyterhubstorage/victor.silva/brazilian-legal-text-bert/Datasets/HelBERT-base-uncased-scratch/teste.txt', 'r') as f:
    dados = [line.strip() for line in f.readlines()]
dados = pd.DataFrame(dados, columns=['text'])
dados

In [ ]:
dados = pd.read_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Objetos/ObjetosWeak/dataset_objects_weak_teste.csv')
dados

In [ ]:
def mask_words(sentence: str) -> list:
    words = sentence.split()
    num_words = len(words)
    palavras_reais = []
    min_masks = max(1, int(num_words * 0.01)) 
    max_masks = max(min_masks, int(num_words * 0.15))
    num_masks = random.randint(min_masks, max_masks)
    masked_words = words.copy()
    if num_masks > 0:
        masked_indices = random.sample(range(num_words), num_masks)
        masked_indices = sorted(masked_indices)
        for index in masked_indices:
            palavras_reais.append(masked_words[index])
            masked_words[index] = '[MASK]'
    masked_sentence = ' '.join(masked_words)
    return masked_sentence, palavras_reais

In [ ]:
def mascaraDataFrame(df:pd.DataFrame, coluna: str) -> pd.DataFrame:
    registros = []
    for linha in tqdm(df.index, desc="Mascarando DataFrame de validação"):
        secao_limpa = df[coluna][linha]
        secao_mascarada, labels = mask_words(secao_limpa)
        registros.append({'sentenca': secao_mascarada, 'labels': labels})
    dataframe = pd.DataFrame(registros)
    return dataframe

In [ ]:
dados = mascaraDataFrame(dados, 'text')
dados

In [ ]:
import re
import string

regex_pontuacao = '[' + re.escape(string.punctuation) + ']'
regex_numero = '((\d+)|(<NUMERO>)|([\-–]))'

In [ ]:
indices_drop = []
for indice in tqdm(dados.index):
    if len(dados['labels'][indice]) > 0:
        for palavra in dados['labels'][indice]:
            if re.search(regex_numero, palavra) or re.search(regex_pontuacao, palavra):
                indices_drop.append(indice)
                break
    else:
        indices_drop.append(indice)
dados = dados.drop(indices_drop).reset_index(drop=True)
dados                

In [ ]:
dados = dados.sample(n=50000).reset_index(drop=True)
dados

In [ ]:
dados.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/MaskedLanguageModeling/dataset_objects_weak_teste_mlm.csv', index=False, encoding="utf8")